In [1]:
"""
Purpose: To Run the proofreading of neurons in the 
test set and measure the precision and recall of the new rules

Segments to look out for that had no axon
864691135012395766 # one of the split segments files not there (split index = 1)
864691135865584005 #soma soma merger
864691136451113215 #it was an inhibitory neuron (part of split)

"""

'\nPurpose: To Run the proofreading of neurons in the \ntest set and measure the precision and recall of the new rules\n\nSegments to look out for that had no axon\n864691135012395766 # one of the split segments files not there (split index = 1)\n864691135865584005 #soma soma merger\n864691136451113215 #it was an inhibitory neuron (part of split)\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-06-01 19:12:17,517 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-01 19:12:17,519 - settings - Setting database.user to celiib
INFO - 2021-06-01 19:12:17,520 - settings - Setting database.password to newceliipass
INFO - 2021-06-01 19:12:17,529 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-06-01 19:12:17,531 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-01 19:12:17,548 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-06-01 19:12:18,063 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-01 19:12:18,066 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-01 19:12:18,067 - settings - Setting database.user to celiib
INFO - 2021-06-01 19:12:18,068 - settings - Setting database.password to newceliipass
INFO - 2021-06-01 19:12:18,069 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-01 19:12:18,070 - settings - Setting database.user to celiib
INFO - 2021-06-01 19:12:18,071 - settings - Setting database.password to newceliipass
INFO - 2021-06-01 19:12:18,073 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-01 19:12:18,611 - settings - Setting enable_python_native_blobs to True


In [4]:
#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-06-01 19:12:18,803 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-01 19:12:18,804 - settings - Setting database.user to celiib
INFO - 2021-06-01 19:12:18,805 - settings - Setting database.password to newceliipass
INFO - 2021-06-01 19:12:18,808 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-01 19:12:19,385 - settings - Setting enable_python_native_blobs to True


In [5]:
import validation_utils as vu
import neuron_visualizations as nviz
import neuron_utils as nru
import proofreading_utils as pru
import time
import system_utils as su

# Getting the Results of the Validation

In [ ]:
pre_source = (minnie.AutoProofreadValidationSegmentMap4() & 
    (dj.U("old_segment_id") & minnie.DecompositionAxon.proj(old_segment_id="segment_id")))

key_source = (pre_source - 
              du.current_validation_segment_id_exclude.proj(old_segment_id="segment_id")
              #& dict(old_segment_id=864691135373402824)
             )
key_source

In [ ]:
#(schema.jobs & "table_name = '__auto_proofread_validation_score4'").delete()

In [ ]:
minnie,_ = du.configure_minnie_vm()
validation_dfs = minnie.AutoProofreadValidationScore4().fetch("validation_df")
validation_dfs = [vu.load_df_val_from_numpy(k) for k in validation_dfs]
validation_dfs

In [ ]:
import pandas_utils as pu
total_validation_df = pu.concat(validation_dfs)
total_validation_df

In [ ]:
vu.synapse_validation_df_to_score_plot(total_validation_df)

# Look at the Neurons That did Poorly

In [ ]:
minnie.AutoProofreadValidationScore4() & 

In [ ]:
bad_seg_ids = (minnie.AutoProofreadValidationSegmentMap4() * (minnie.AutoProofreadValidationScore4() & "pre_precision < 0.5" & "pre_fp > 15")).fetch("segment_id")
bad_seg_ids

In [ ]:
segment_id = bad_seg_ids[4]
curr_table = (minnie.AutoProofreadValidationScore4() & dict(segment_id=segment_id))
curr_table

In [ ]:
segment_id = bad_seg_ids[4]
curr_table = (minnie.AutoProofreadValidationScore4() & dict(segment_id=segment_id))
decomp_path,validation_df = curr_table.fetch1("decomposition","validation_df")
neuron_obj = du.filepath_to_neuron_obj(decomp_path)

In [ ]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")

In [ ]:
return_validation_df_revised = vu.load_df_val_from_numpy(validation_df)
filt_neuron = neuron_obj
vu.plot_validation_synapses(filt_neuron.segment_id,
                            validation_df = return_validation_df_revised,
                             synapse_dict = None,
                             synapses_type_to_plot = None,
                             synapses_type_to_not_plot = None,
                             verbose=False,
                             synapse_type = "presyn",
                             TP_color="yellow",
                            TN_color="aqua",
                            FP_color="black",
                            FN_color="orange",
                             synapse_scatter_size = 0.05,
                             
                             #for plotting the actual mesh parts to go along
                             
                             neuron_obj = filt_neuron,
                             plot_only_axon_skeleton = True,
                             error_mesh_color = "red",
                             valid_mesh_color = "green",
                             valid_skeleton_color = "black",
                             mesh_alpha = 0.3,)

In [ ]:
import networkx_utils as xu
xu.upstream_node(neuron_obj[0].concept_network_directional,70)

In [ ]:
neuron_obj[0][86].skeletal_length

In [ ]:
nviz.plot_limb_branch_dict(neuron_obj,
                          {'L0': np.array([ 70])})

In [ ]:
neuron_obj[0][70].width_new

In [ ]:
axon_comparison_distance = 1000,

In [ ]:
import neuron_searching as ns
result_limb_branch = ns.query_neuron(neuron_obj,
    functions_list=["parent_angle","upstream_skeletal_length","skeletal_length","axon_width"],
    query="(parent_angle>110) and (skeletal_length>5000) and (upstream_skeletal_length>1000) and (axon_width>105)",
        function_kwargs=dict(comparison_distance=1000),
    return_dataframe=False,
    limbs_to_process=[neuron_obj.axon_limb_name])

print(f"result_limb_branch = {result_limb_branch}")
nviz.plot_limb_branch_dict(neuron_obj,
                           result_limb_branch)

In [ ]:
nviz.plot_limb_branch_dict(neuron_obj,
                          {"L0":[17,23]})

In [ ]:
xu.downstream_nodes(curr_limb.concept_network_directional,
                   17)

In [ ]:
import neuron_utils as nru
curr_limb = neuron_obj[0]
parent_angle = nru.find_parent_child_skeleton_angle(curr_limb,
                                            23,
                                                    comparison_distance=1000,
                                            plot_extracted_skeletons=True)
parent_angle

In [ ]:
xu.upstream_node(curr_limb.concept_network_directional,23)

In [ ]:
bs = [17,23]
c = ["red","blue"]
nviz.plot_objects(meshes=[curr_limb[k].mesh for k in bs],
                 meshes_colors=c,
                 skeletons =[curr_limb[k].skeleton for k in bs],
                 skeletons_colors=c)

In [ ]:
nviz.plot_branches_with_spines(neuron_obj[0][17])

In [ ]:
nviz.plot_branches_with_spines(neuron_obj[0][23])

In [ ]:
nviz.plot_limb_branch_dict(neuron_obj,
                           result_limb_branch)

# Trying Out the New Error Function

In [ ]:
curr_table = (minnie.AutoProofreadValidationScore4() & dict(segment_id=segment_id))
curr_table*minnie.AutoProofreadValidationSegmentMap4()

In [ ]:
decomp_path = (minnie.DecompositionAxon() & dict(segment_id=864691135462735678,
                                  split_index = 0
                                 )).fetch1("decomposition")
pre_filt_neuron = du.filepath_to_neuron_obj(decomp_path)

In [ ]:
pre_filt_neuron

In [ ]:
nviz.visualize_neuron(pre_filt_neuron,
                     limb_branch_dict="all")

In [ ]:
(filt_neuron,
             return_synapse_df_revised,
             return_synapse_df_errors,
            return_validation_df_revised,
            return_validation_df_extension) =  vu.filtered_neuron_score(neuron_obj = pre_filt_neuron,   
                                filter_list = [pru.exc_double_back_and_width_change_filter],
                                plot_limb_branch_filter_with_disconnect_effect = True,
                                verbose = True,
                                plot_score=False,
                                nucleus_id = 560530,
                                return_synapse_df_errors=True,
                                return_validation_df_extension = True,                                        
                                split_index=0)

# Trying on the ALREADY Proofread Neuron

In [ ]:
segment_id = bad_seg_ids[4]
curr_table = (minnie.AutoProofreadValidationScore4() & dict(segment_id=segment_id))
decomp_path,validation_df = curr_table.fetch1("decomposition","validation_df")
neuron_obj = du.filepath_to_neuron_obj(decomp_path)

In [ ]:
neuron_obj[0][70]

In [ ]:
(filt_neuron,
             return_synapse_df_revised,
             return_synapse_df_errors,
            return_validation_df_revised,
            return_validation_df_extension) =  vu.filtered_neuron_score(neuron_obj = pre_filt_neuron,   
                                filter_list = [pru.exc_double_back_and_width_change_filter],
                                plot_limb_branch_filter_with_disconnect_effect = True,
                                verbose = True,
                                plot_score=False,
                                nucleus_id = 560530,
                                return_synapse_df_errors=True,
                                return_validation_df_extension = True,                                        
                                split_index=0)